In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import PassiveAggressiveClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

df_true = pd.read_csv('True.csv')
df_fake = pd.read_csv('Fake.csv')

df_true['label'] = 1
df_fake['label'] = 0

df = pd.concat([df_true, df_fake]).reset_index(drop=True)
df = df.sample(frac=1).reset_index(drop=True)

df = df[['text', 'label']]
print(f"Dataset loaded: {df.shape[0]} articles")
df.head()

Dataset loaded: 44898 articles


,text,label
0,TOKYO (Reuters) - Japan s navy is supplying fu...,1
1,Astute CBS reporter Rebecca Shabad couldn t po...,0
2,"MEDIA SILENT! Twin Falls, Idaho Police Investi...",0
3,NEW YORK/WELLINGTON (Reuters) - Erik Duhaime i...,1
4,"Across the nation, thousands of upset American...",0


In [ ]:
x_train, x_test, y_train, y_test = train_test_split(df['text'], df['label'], test_size=0.2, random_state=42)

tfidf_vectorizer = TfidfVectorizer(stop_words='english', max_df=0.7)

tfidf_train = tfidf_vectorizer.fit_transform(x_train)
tfidf_test = tfidf_vectorizer.transform(x_test)

pac = PassiveAggressiveClassifier(max_iter=50)
pac.fit(tfidf_train, y_train)
print("Model training complete.")

Model training complete.


In [ ]:
y_pred = pac.predict(tfidf_test)

score = accuracy_score(y_test, y_pred)
print(f'Final Accuracy: {round(score*100,2)}%')

print("\nClassification Report:")
print(classification_report(y_test, y_pred, target_names=['Fake', 'Real']))

Final Accuracy: 99.49%

Classification Report:
              precision    recall  f1-score   support

        Fake       1.00      0.99      1.00      4679
        Real       0.99      1.00      0.99      4301

    accuracy                           0.99      8980
   macro avg       0.99      0.99      0.99      8980
weighted avg       0.99      0.99      0.99      8980



In [ ]:
import ipywidgets as widgets
from IPython.display import display, clear_output

# Define the UI
text_input = widgets.Textarea(
    placeholder='Paste news here...',
    description='News:',
    layout={'width': '100%', 'height': '150px'}
)
check_button = widgets.Button(description='Check Authenticity', button_style='info')
out_box = widgets.Output()

def perform_check(b):
    with out_box:
        clear_output()
        print("Processing... please wait.") 
        
        try:
            raw_text = text_input.value.strip()
            if not raw_text:
                print("❌ Error: The text box is empty!")
                return
            
            transformed = tfidf_vectorizer.transform([raw_text])
            prediction = pac.predict(transformed)
            
            clear_output()
            if prediction[0] == 1:
                print("✅ RESULT: THIS NEWS IS REAL")
            else:
                print("🚨 RESULT: THIS NEWS IS FAKE")
        except NameError:
            print("❌ Error: 'tfidf_vectorizer' or 'pac' not found. Please run the training cells first!")
        except Exception as e:
            print(f"❌ An unexpected error occurred: {e}")

check_button.on_click(perform_check)

display(text_input, check_button, out_box)

Textarea(value='', description='News:', layout=Layout(height='150px', width='100%'), placeholder='Paste news h…

Button(button_style='info', description='Check Authenticity', style=ButtonStyle())

Output()